In [ ]:
using JuMP, Gurobi
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60) 

#placeholder vars
#plant_locs - 300x4 mtx: x, y coordinates of plants, size (unknown units), and year of expiration
##waste_locs - x,y coords of potential waste locations
#fixed_cost: fixed price of operating any facility
#variable_cost: scales linearly with size (undetermined units)
#distance_mtx: 95x300 mtx of distances from waste plant x to solar facility y
#transportation_cost: cost of physically transporting 1 unit of waste 1 unit of distance

@variable(model, loc_made[1:95, 1:50], Bin) #next 50 years (is this appropriate?)
@variable(model, loc_size[1:95] .>= 0) #assuming size of waste facility doesn't change; right now same units as waste
@variable(model, solar_to_loc[1:300, 1:95, 1:50]) #assuming 300 solar locations in MA
@variable(model, facility_exists[1:95], Bin) 


@constraint(model, [i=1:95,t=1:50], facility_exists[i] >= loc_made[i, t])
@constraint(model, [i=1:95],facility_exists[i] <= sum(loc_made[i, t] for t in 1:50))

M = 1000000
for i in 1:95
    @constraint(model, loc_size[i] <= M * facility_exists[i])
    @constraint(model, M*loc_size[i] >= facility_exists[i])
end


delta = .99 #1% decay in price of both operation and creation by year (subject to change)

year_exps = []
for i in 1:300
    push!(year_exps, plant_locs[i, 3] - 1989) #assuming first expires in 1990, subject to change
end

for year in 1:50
    if year != year_exps[i]
        @constraint(model, [i = 1:300, j=1:95], solar_to_loc[i, j, year] == 0)
    else
        @constraint(model, [i = 1:300], plant_locs[i, 3] == sum(solar_to_loc[i, j, year] for j in 1:95))
        @constraint(model, [i= 1:300, j=1:95], solar_to_loc[i,j,year] .<= loc_size[j]) #same units as waste
    end
end

@variable(model, started_by_year[1:95, 1:50], Bin) #becomes 1 when first starts
@constraint(model, [i=1:95], started_by_year[i, 1] == loc_made[i, 1])
@constraint(model, [i=1:95, t=2:50], started_by_year[i, t] >= started_by_year[i, t-1])
@constraint(model, [i=1:95, t=2:50], started_by_year[i, t] >= loc_made[i, t])
@constraint(model, [i=1:95, t=2:50], started_by_year[i, t] <= started_by_year[i, t-1] + loc_made[i, t])
@variable(model, first_year_made[1:95] >= 0) #first year facility starts operating
@constraint(model, [i=1:95], first_year_made[i] == sum(t * (started_by_year[i, t] - (t == 1 ? 0 : started_by_year[i, t-1])) for t=1:50))

@objective(model, Min, sum(delta^first_year_made[i]*fixed_cost + sum(loc_made[i,j]*delta^j*loc_size[i]*variable_cost + sum(solar_to_loc[s, i, j]*distance_mtx[i,s]*transportation_cost for s in 1:300) for j in 1:50) + for i in 1:95)) 

optimize!(model)
termination_status(model)